# 언어 모델에서 대화란 무엇인가

## HumanMessage와 AIMessage를 번갈아 가며 대화한다

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage


chat = ChatOpenAI(
    model='gpt-3.5-turbo'
)

result = chat.invoke(
    [
        HumanMessage(content="계란찜을 만드는 재료를 알려주세요.")
    ]
)

print(result.content)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage


chat = ChatOpenAI(
    model='gpt-3.5-turbo'
)

result = chat.invoke(
    [
        HumanMessage(content="계란찜을 만드는 재료를 알려주세요."),
        AIMessage(content="""
계란찜을 만드는데 필요한 재료는 다음과 같습니다.

1. 계란 - 4개
2. 물 - 1컵
3. 소금 - 1/2 작은술
4. 설탕 - 1/2 작은술
5. 다진파 - 약간 (선택사항)
6. 다진마늘 - 약간 (선택사항)
7. 소금 - 약간 (선택사항)
8. 후추 - 약간 (선택사항)        
        
        """),
        HumanMessage(content="위의 답변을 영어로 번역하세요")
    ]
)

print(result.content)

# 문맥에 맞는 답변을 할 수 있는 챗봇 만들기

## Chat models로 대화 기록을 기반으로 한 응답을 하게 하는 것

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True
)
memory.save_context(
    {
        "input":"안녕하세요!"
    },
    {
        "output":"안녕하세요! 잘 지내고 계신가요? 궁금한 점이 있으면 알려주세요. 어떻게 도와 드릴까요"
    }
)
memory.save_context(
    {
        "input":"오늘 날씨가 좋네요"
    },
    {
        "output":"저는 AI이기 때문에 실제 날씨를 느낄 수는 없지만, 날씨가 좋은 날은 외출이나 활동을 즐기기에 좋은 날 입니다."
    }
)

print(memory.load_memory_variables({}))

In [ ]:
import chainlit as cl
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)

memory = ConversationBufferMemory(
    return_messages=True
)

chain = ConversationChain(
    memory = memory,
    llm = chat
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="저는 대화의 맥락을 고려해 답변을 할 수 있는 채팅 봇 입니다. 메시지를 입력해 주세요.").send()

@cl.on_message
async def on_message(input_message):
    message = input_message.content
    result = chain(message)
   
    await cl.Message(content=result['response']).send()


# 히스토리를 데이터베이스에 저장하고 영속화하기

In [ ]:
import os
import chainlit as cl
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import RedisChatMessageHistory
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)

history = RedisChatMessageHistory(
    session_id="chat_history",
    url=os.environ.get('REDIS_URL'),

)

memory = ConversationBufferMemory(
    return_messages=True,
    chat_memory=history
)

chain = ConversationChain(
    memory = memory,
    llm = chat
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="저는 대화의 맥락을 고려해 답변을 할 수 있는 채팅 봇 입니다. 메시지를 입력해 주세요.").send()

@cl.on_message
async def on_message(input_message):
    message = input_message.content
    result = chain(message)
   
    await cl.Message(content=result['response']).send()


# 여러 개의 대화 기록을 가질 수 있는 챗봇 만들기

## 세션 ID를 바꿔서 대화 기록 전환하기

In [ ]:
import os
import chainlit as cl
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory, RedisChatMessageHistory
from langchain.schema import HumanMessage

chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)

@cl.on_chat_start
async def on_chat_start():
    thread_id = None
    while not thread_id:
        res = await cl.AskUserMessage(content="저는 대화의 맥락을 고려해 답변을 할 수 있는 채팅 봇 입니다. 스레드 ID를 입력하세요.", timeout=600).send()
        if res:
            print(res)
            thread_id = res['output']

    history = RedisChatMessageHistory(
        session_id=thread_id,
        url=os.environ.get('REDIS_URL'),

    )

    memory = ConversationBufferMemory(
        return_messages=True,
        chat_memory=history
    )

    chain = ConversationChain(
        memory = memory,
        llm = chat
    )

    memory_message_result = chain.memory.load_memory_variables({})
    
    messages = memory_message_result['history']

    for message in messages:
        if isinstance(message, HumanMessage):
            await cl.Message(
                author='User',
                content=f'{message.content}'
            ).send()
        else:
            await cl.Message(
                author='ChatBot',
                content=f'{message.content}'
            ).send()
    cl.user_session.set('chain', chain)


@cl.on_message
async def on_message(input_message):
    message = input_message.content
    chain = cl.user_session.get('chain')
    result = chain(message)
   
    await cl.Message(content=result['response']).send()


# 매우 긴 대회 기록에 대응한다

## 대화를 요약해 토큰 수 제한에 대응

In [ ]:
import os
import chainlit as cl
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import RedisChatMessageHistory
from langchain.memory import ConversationSummaryMemory

chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)

history = RedisChatMessageHistory(
    session_id="chat_history",
    url=os.environ.get('REDIS_URL'),

)

memory = ConversationSummaryMemory(
    llm=chat,
    return_messages=True,
    chat_memory=history
)

chain = ConversationChain(
    memory = memory,
    llm = chat
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="저는 대화의 맥락을 고려해 답변을 할 수 있는 채팅 봇 입니다. 메시지를 입력해 주세요.").send()

@cl.on_message
async def on_message(input_message):
    message = input_message.content
    messages = chain.memory.load_memory_variables({})['history']
    print(f'저장된 메시지 개수 : {len(messages)}')
    for saved_message in messages:
        print(saved_message.content)
        
    result = chain(message)
   
    await cl.Message(content=result['response']).send()
